In [2]:
import pandas as pd
import spline
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
from merge_script import modify_first_column
import math
from sklearn.decomposition import PCA
import glob
import Calculate_Feature
import os
from spline import np_move_avg

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score
import glob
from sklearn.model_selection import train_test_split, KFold, cross_val_score, LeaveOneOut
from sklearn.manifold import TSNE
from sklearn import svm, tree
import warnings
warnings.filterwarnings("ignore")


def use_pca(featuer_data, n_components=5):
    pca = PCA(n_components)
    axis_fea = pca.fit_transform(featuer_data)  # 每个样本降为n_components维
    fea = []
    for raw in range(axis_fea.shape[0]):
        for ele in axis_fea[raw, :]:
            fea.append(ele)
    return fea


def axis_normalization(datalist, stage=1):
    normalized_data = []
    #datalist = np.array(datalist)
    for data in datalist:
        maximum = max(data)
        minimum = min(data)
        # print("maximum is:",maximum,"minimum is:",minimum)
        normalized_data.append([(item - minimum) / (maximum - minimum) * stage for item in data])
    return np.array(normalized_data)


def three_fuse(data):
    axis_num = int(data.shape[0] / 3)
    f_nd = []

    for f_in in range(axis_num):
        nd = []
        index = f_in * 3
        for i in range(data.shape[1]):
            d = data[:, i]
            fuse_data = math.sqrt(d[index] * d[index] + d[index + 1] * d[index + 1] + d[index + 2] * d[index + 2])
            nd.append(fuse_data)
        f_nd.append(nd)
    return f_nd

In [13]:
# l_rf = RandomForestClassifier(n_estimators=100, random_state=42)
l_rf = RandomForestClassifier(n_estimators=100, random_state=42)
l_svc = svm.SVC(C=1.0, kernel='rbf')
# l_svc = svm.SVC(C=1000, kernel='rbf')
l_clftree = tree.DecisionTreeClassifier(criterion='entropy',random_state=42)

# loo = LeaveOneOut()

real_path = 'save_data/12-16-Realdataset-Xia/Dataset-ID-'

ID = [1, 2, 3, 4, 5, 6, 7]
label = [0, 1, 2, 3, 4, 5, 6, 7]


# Real_data_path = "save_data/RealData_Seg_byName_120/"
Real_data_path = "save_data/RealData_Seg_byName/"

virtual_data_path = "save_data/Pos_New_New_Virtual_Seg_byScale_xy_spline5_120/" # awsome !!!
virtual_data_path_2 = "save_data/Pos_New_New_Virtual_Seg_byScale_xy_18_spline3_60/" # awsome !!!

# virtual_data_path_2 = "save_data/Virtual_New_Conv_0108_size7_spline3_240/"
virtual_data_path_3 = "save_data/Virtual_New_Conv_0108_size7_spline4_240/"
# virtual_data_path_4 = "save_data/Virtual_New_Conv_0108_size7_spline5_240/"

virtual_data_path_4 = ["save_data/Virtual_New_Conv_0108_size3_spline3_240/", "save_data/Virtual_New_Conv_0108_size3_spline5_240/", "save_data/Virtual_New_Conv_0108_size3_spline7_240/"]

virtual_data_path_5 = ["save_data/Virtual_New_Conv_0108_size5_spline3_240/", "save_data/Virtual_New_Conv_0108_size5_spline5_240/", "save_data/Virtual_New_Conv_0108_size5_spline7_240/"]

virtual_data_path_6 = ["save_data/Virtual_New_Conv_0108_size7_spline3_240/", "save_data/Virtual_New_Conv_0108_size7_spline5_240/", "save_data/Virtual_New_Conv_0108_size7_spline7_240/"]





virtual_data_path_7 = ["save_data/Virtual_New_Conv_0108_concave_spline3_240/","save_data/Virtual_New_Conv_0108_concave_spline4_240/","save_data/Virtual_New_Conv_0108_concave_spline5_240/"]
virtual_data_path_8 = ["save_data/Virtual_New_Conv_0108_convex_spline3_240/","save_data/Virtual_New_Conv_0108_convex_spline4_240/","save_data/Virtual_New_Conv_0108_convex_spline5_240/"]
virtual_data_path_9 = ["save_data/Virtual_New_Conv_0108_inc_spline3_240/","save_data/Virtual_New_Conv_0108_inc_spline4_240/","save_data/Virtual_New_Conv_0108_inc_spline5_240/"]
virtual_data_path_10 = ["save_data/Virtual_New_Conv_0108_dec_spline3_240/","save_data/Virtual_New_Conv_0108_dec_spline4_240/","save_data/Virtual_New_Conv_0108_dec_spline5_240/"]

virtual_data_path_11 = ["save_data/Virtual_New_Conv_1230_spline3_240/","save_data/Virtual_New_Conv_1230_spline4_240/","save_data/Virtual_New_Conv_1230_spline5_240/"]

virtual_data_path_12 = ["save_data/Virtual_New_Conv_1231_spline3_240/","save_data/Virtual_New_Conv_1231_spline4_240/","save_data/Virtual_New_Conv_1231_spline5_240/"]

virtual_data_path_size3 = ["CorData-seg/Virtual_New_Conv_0115_size3_spline3_240/","CorData-seg/Virtual_New_Conv_0115_size3_spline4_240/","CorData-seg/Virtual_New_Conv_0115_size3_spline5_240/"]

virtual_data_path_size3_t = ["CorData-Seg/Virtual_New_Conv_0115_size3_trans_spline3_240/","CorData-Seg/Virtual_New_Conv_0115_size3_trans_spline4_240/","CorData-Seg/Virtual_New_Conv_0115_size3_trans_spline5_240/"]


virtual_data_path_size5 = ["CorData-seg/Virtual_New_Conv_0115_size5_spline3_240/","CorData-seg/Virtual_New_Conv_0115_size5_spline4_240/","CorData-seg/Virtual_New_Conv_0115_size5_spline5_240/"]

virtual_data_path_size5_t = ["CorData-Seg/Virtual_New_Conv_0115_size5_trans_spline3_240/","CorData-Seg/Virtual_New_Conv_0115_size5_trans_spline4_240/","CorData-Seg/Virtual_New_Conv_0115_size5_trans_spline5_240/"]


virtual_data_path_size7 = ["CorData-seg/Virtual_New_Conv_0115_size7_spline3_240/","CorData-seg/Virtual_New_Conv_0115_size7_spline4_240/","CorData-seg/Virtual_New_Conv_0115_size7_spline5_240/"]

virtual_data_path_size7_t = ["CorData-Seg/Virtual_New_Conv_0115_size7_trans_spline3_240/","CorData-Seg/Virtual_New_Conv_0115_size7_trans_spline4_240/","CorData-Seg/Virtual_New_Conv_0115_size7_trans_spline5_240/"]


virtual_data_path_scale_t = ["CorData-Seg/scale_0123_trans_spline3_240/", "CorData-Seg/scale_0123_trans_spline4_240/", "CorData-Seg/scale_0123_trans_spline5_240/"]
virtual_data_path_scale = ["CorData-Seg/scale_0123_spline3_240/", "CorData-Seg/scale_0123_pline4_240/", "CorData-Seg/scale_0123_spline5_240/"]

ori_data_path_2 = "save_data/Animation_record_pos_data_spline5_120"
# ori_data_path_2 = "save_data/Ori_motion_pos_data_spline5_120"

ori_data_path = 'save_data/Conv_Motion_acc_data_spline3_240/'



virtual_motion_type = ['ankleTap', 'highKnee', 'reverseLunge', 'sideCrunch', 'sidetoside']

motion_name = ['ankle', 'highKnee', 'Kneekick', 'reverseLunge', 'sideCrunch', 'sidetoside', 'warmup']

human_name = ["Effy", "Leafy", "Nick", "Qin", "Tonii", "Xu", "Yamamoto"]
# human_name = ["Effy", "Leafy", "Nick", "Qin", "Xu", "Yamamoto"]
# scale_list = ["0.8", "0.9", "1.1","1.3", "1.2"]
# scale_list = ["0.5", "0.7", "0.8","0.9", "1.1", "1.2","1.3", "1.5"]
scale_list = ["0.5", "0.7", "0.9", "1.1", "1.3", "1.5"]
scale_list_2 = ["0.5", "0.7", "1.3", "1.5"]
# scale_list_2 = ["0.5", "1.5"]
# scale_list = ["0.7", "0.9", "1.1", "1.3", "1.5"]
scale_list_conv = ["0", "1", "2", "3"]
scale_list_conv_2 = ["0", "1", "2"]
scale_list_conv_3 = ["0", "1", "2", "3", "4"]

virtual_total_file = []
real_total_file = []
total_file = []

# amplitude
# for scale in scale_list:
#     virtual_total_file.append(virtual_data_path + scale)
#     total_file.append(virtual_data_path + scale)
# 
# for scale in scale_list_conv:
#     virtual_total_file.append(virtual_data_path_2 + scale)
#     total_file.append(virtual_data_path_2 + scale)
# 
# for scale in scale_list_conv:
#     virtual_total_file.append(virtual_data_path_3 + scale)
#     total_file.append(virtual_data_path_3 + scale)
# 
# for scale in scale_list_conv:
#     virtual_total_file.append(virtual_data_path_4 + scale)
#     total_file.append(virtual_data_path_4 + scale)
# 
# 
# for path in virtual_data_path_4:
#     for scale in scale_list_conv:
#             virtual_total_file.append(path + scale)
#             total_file.append(path + scale)
# 
# for path in virtual_data_path_5:
#     for scale in scale_list_conv:
#             virtual_total_file.append(path + scale)
#             total_file.append(path + scale)
# 
# for path in virtual_data_path_6:
#     for scale in scale_list_conv:
#             virtual_total_file.append(path + scale)
#             total_file.append(path + scale)





# for scale in scale_list_conv:
#     for path in virtual_data_path_7:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)
    
# for scale in scale_list_conv:
#     for path in virtual_data_path_8:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)

# for scale in scale_list_conv:
#     for path in virtual_data_path_9:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)
# 
# for scale in scale_list_conv:
#     for path in virtual_data_path_10:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)
# 
# for scale in scale_list_conv:
#     for path in virtual_data_path_11:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)

# for scale in scale_list_conv:
#     for path in virtual_data_path_size3:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)
# 
# for scale in scale_list_conv:
#     for path in virtual_data_path_size3_t:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)
# 
# for scale in scale_list_conv:
#     for path in virtual_data_path_size5:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)
# 
# for scale in scale_list_conv:
#     for path in virtual_data_path_size5_t:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)
# 
# for scale in scale_list_conv:
#     for path in virtual_data_path_size7:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)
# 
# for scale in scale_list_conv:
#     for path in virtual_data_path_size7_t:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)
# 0 -  24 - Factor
for scale in scale_list_2:
    for path in virtual_data_path_scale:
        virtual_total_file.append(path + scale)
        total_file.append(path + scale)

for scale in scale_list_2:
    for path in virtual_data_path_scale_t:
        virtual_total_file.append(path + scale)
        total_file.append(path + scale)

# ori
total_file.append(ori_data_path_2)
# total_file.append(ori_data_path)
print(len(total_file))
# for index in ID:
#     real_total_file.append(real_path + str(index))
#     total_file.append(real_path + str(index))

for name in human_name:
    real_total_file.append(Real_data_path + name)
    total_file.append(Real_data_path + name)
print(total_file)

print(len(total_file))

25
['CorData-Seg/scale_0123_spline3_240/0.5', 'CorData-Seg/scale_0123_pline4_240/0.5', 'CorData-Seg/scale_0123_spline5_240/0.5', 'CorData-Seg/scale_0123_spline3_240/0.7', 'CorData-Seg/scale_0123_pline4_240/0.7', 'CorData-Seg/scale_0123_spline5_240/0.7', 'CorData-Seg/scale_0123_spline3_240/1.3', 'CorData-Seg/scale_0123_pline4_240/1.3', 'CorData-Seg/scale_0123_spline5_240/1.3', 'CorData-Seg/scale_0123_spline3_240/1.5', 'CorData-Seg/scale_0123_pline4_240/1.5', 'CorData-Seg/scale_0123_spline5_240/1.5', 'CorData-Seg/scale_0123_trans_spline3_240/0.5', 'CorData-Seg/scale_0123_trans_spline4_240/0.5', 'CorData-Seg/scale_0123_trans_spline5_240/0.5', 'CorData-Seg/scale_0123_trans_spline3_240/0.7', 'CorData-Seg/scale_0123_trans_spline4_240/0.7', 'CorData-Seg/scale_0123_trans_spline5_240/0.7', 'CorData-Seg/scale_0123_trans_spline3_240/1.3', 'CorData-Seg/scale_0123_trans_spline4_240/1.3', 'CorData-Seg/scale_0123_trans_spline5_240/1.3', 'CorData-Seg/scale_0123_trans_spline3_240/1.5', 'CorData-Seg/sca

In [14]:
class CustomLeaveOneOut:
    def __init__(self, start=7, end=13):
        self.start = start
        self.end = end
        self.current = start

    def __iter__(self):
        return self

    def __next__(self):
        if self.current > self.end:
            raise StopIteration
        test_indices = [self.current]
        train_indices = list(range(self.start, self.end + 1))
        train_indices.remove(self.current)
        self.current += 1
        # return list(range(47)) + train_indices, test_indices
        # return train_indices, test_indices

        # return list(range(12))+ list(range(36,48)), test_indices

        # return list(range(3,5)) + list(range(1)) + list(range(6,7)), test_indices
        # return train_indices, test_indices
        return list(range(24, 25)), test_indices

# 0 -  24 - Factor
# for scale in scale_list_2:
#     for path in virtual_data_path_scale:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)
# 
# for scale in scale_list_2:
#     for path in virtual_data_path_scale_t:
#         virtual_total_file.append(path + scale)
#         total_file.append(path + scale)


# loo = CustomLeaveOneOut(start=6, end=11)
#
# for train_indices, test_indices in loo:
#     print("Train Indices:", train_indices)
#     print("Test Indices:", test_indices)


Real_motion_type = ['ReverseLunge','HighKnee','sidetoside'] # 相同awsome!!!!!!!!!!!

encode_feauture = [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
# encode_feauture = [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]

In [15]:
# loo = CustomLeaveOneOut(start=1, end=7)
# loo = CustomLeaveOneOut(start=7, end=13)
# loo = CustomLeaveOneOut(start=14, end=20)
# loo = CustomLeaveOneOut(start=18, end=24) 
loo = CustomLeaveOneOut(start=25, end=31)
rf_predicted = []
svm_predicted = []
decisiontree_predicted = []
label_save_list = []
predict_SVM_save_list = []
predict_RF_save_list = []
predict_DT_save_list = []


# for train_index, test_index in loo.split(total_file):
#     print(train_index,test_index)
for train_index, test_index in loo:
    print(train_index,test_index)
    Data_X = []
    Label_X = []
    TestData_X = []
    TestLabel_X = []
    true_label = []
    for tmtrain in train_index:
        # print(tmtrain)
        subject = total_file[tmtrain]  # for each subject
        # print(subject)
        #subject_file = glob.glob(os.path.join(subject,'*.csv'))
        for i in range(len(Real_motion_type)):
            motion = Real_motion_type[i]
            motion_file = glob.glob(os.path.join(subject + '/' + '*' + motion + '*', '*.csv'))
            # motion_file = os.path.join(subject + '/' + 'motion', '*.csv')
            # motion_file = glob.glob(os.path.join(subject + '/' + 'motion/', '*.csv'))
            # print(subject + '/' +  motion )
            # print(motion_file)
            for motion_frame in motion_file:
                #print(motion_frame)
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [3,4,5,6,7,8,9,10,11,12,13,14])
                # df = pd.read_csv(motion_frame, index_col=False, header=None, usecols= [9,10,11,12,13,14])#1,2,3,4,5,6,7,8,9,10,11,12
                df = pd.read_csv(motion_frame, index_col=False, header=None, usecols= [9,10,11])#0,1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None, usecols= [9])#0,1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [12,13,14])  #1,2,3,4,5,6,7,8,9,10,11,12
                df = np.array(df).T


                # timestamp = []
                # for t in range(df.shape[1]):
                #     timestamp.append(t*0.05)
                # timestamp = np.array(timestamp)

                # for k in range(df.shape[0]):
                #     df[k, :] = np_move_avg(df[k, :], 15, mode='same')

                # fuse three axis-data
                fuse_df = three_fuse(df)
                nor_df = axis_normalization(fuse_df)  #.tolist()
                # nor_df = axis_normalization(df)  #.tolist()

                axis_fea = []
                # rd = []
                # print(nor_df.shape)
                for raw in range(nor_df.shape[0]):
                # for raw in range(df.shape[0]):
                    tmd = nor_df[raw, :]

                    # resampling = spline.spline_cal(timestamp,tmd.tolist(),10)
                    #
                    # tmd = resampling.resample()
                    # print(tmd)

                    # tmd = df[raw, :]
                    # rd.append(tmd)
                    cal_fea = Calculate_Feature.Get_Feature(tmd, encode_feauture)
                    fea = cal_fea.cal_result()
                    for f in fea:
                        axis_fea.append(f)

                # print(axis_fea)

                Data_X.append(axis_fea)
                Label_X.append(label[i])

                # print(Label_X)

    Real_Data = np.array(Data_X)
    Real_Label = np.array(Label_X)
    # print(Real_Data.shape)
    # print(Real_Label.shape)

    # for i in range(Real_Data.shape[0]):
    #     Real_Data[i] = [0 if math.isnan(x) else x for x in Real_Data[i]]
    # print("real sample size before PCA is:", Real_Data.shape)
    #
    # pca = PCA(2)
    # Real_Data = pca.fit_transform(Real_Data)

    # tsne = TSNE(2)
    # Real_Data = tsne.fit_transform(Real_Data)

    print("real sample size is:", Real_Data.shape)
    l_rf.fit(Real_Data, Real_Label)
    l_svc.fit(Real_Data, Real_Label)
    l_clftree.fit(Real_Data, Real_Label)

    for tmtest in test_index:
        # print(tmtest)
        subject = total_file[tmtest]  # for each subject
        print(subject)
        #subject_file = glob.glob(os.path.join(subject,'*.csv'))
        for i in range(len(Real_motion_type)):
            motion = Real_motion_type[i]
            motion_file = glob.glob(os.path.join(subject + '/' + '*' + motion + '*', '*.csv'))
            for motion_frame in motion_file:
                #print(motion_frame)
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [3,4,5,6,7,8,9,10,11,12,13,14])  #1,2,3,4,5,6,7,8,9,10,11,12
                df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [9,10,11])  #1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [9])  #1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [12,13,14])  #1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None,  usecols= [9,10,11,12,13,14])  #1,2,3,4,5,6,7,8,9,10,11,12
                # df = pd.read_csv(motion_frame, index_col=False, header=None)  #1,2,3,4,5,6,7,8,9,10,11,12
                df = np.array(df).T


                # timestamp = []
                # for t in range(df.shape[1]):
                #     timestamp.append(t*0.05)
                # timestamp = np.array(timestamp)

                #
                # for k in range(df.shape[0]):
                #     df[k, :] = np_move_avg(df[k, :], 20, mode='same')

                #fuse three axis-data
                fuse_df = three_fuse(df)
                nor_df = axis_normalization(fuse_df)  #.tolist()
                # nor_df = axis_normalization(df)  #.tolist()

                axis_fea = []
                # rd = []
                for raw in range(nor_df.shape[0]):
                # for raw in range(df.shape[0]):
                    tmd = nor_df[raw, :]
                    # tmd = df[raw, :]


                    # resampling = spline.spline_cal(timestamp,tmd.tolist(),10)
                    #
                    # tmd = resampling.resample()


                    # rd.append(tmd)
                    cal_fea = Calculate_Feature.Get_Feature(tmd, encode_feauture)
                    fea = cal_fea.cal_result()
                    for f in fea:
                        axis_fea.append(f)



                TestData_X.append(axis_fea)
                TestLabel_X.append(label[i])


    TestData = np.array(TestData_X)

    # for i in range(TestData.shape[0]):
    #     TestData[i] = [0 if math.isnan(x) else x for x in TestData[i]]

    TestLabel = np.array(TestLabel_X)
    label_save_list.append(TestLabel)

    # print("test sample size before PCA is:", TestData.shape)

    # pca = PCA(2)
    # TestData = pca.fit_transform(TestData)

    # tsne = TSNE(2)
    # TestData = tsne.fit_transform(TestData)

    print("test sample size is:", TestData.shape)


    predict_RF_save_list.append(l_rf.predict(TestData))
    predict_SVM_save_list.append(l_svc.predict(TestData))
    predict_DT_save_list.append(l_clftree.predict(TestData))


    rf_predicted.append(accuracy_score(TestLabel, l_rf.predict(TestData)))
    svm_predicted.append(accuracy_score(TestLabel, l_svc.predict(TestData)))
    decisiontree_predicted.append(accuracy_score(TestLabel, l_clftree.predict(TestData)))

    #true_label.append(TestLabel_X[0])

    print("*****")

[24] [25]
real sample size is: (73, 9)
save_data/RealData_Seg_byName/Effy
test sample size is: (66, 9)
*****
[24] [26]
real sample size is: (73, 9)
save_data/RealData_Seg_byName/Leafy
test sample size is: (66, 9)
*****
[24] [27]
real sample size is: (73, 9)
save_data/RealData_Seg_byName/Nick
test sample size is: (66, 9)
*****
[24] [28]
real sample size is: (73, 9)
save_data/RealData_Seg_byName/Qin
test sample size is: (66, 9)
*****
[24] [29]
real sample size is: (73, 9)
save_data/RealData_Seg_byName/Tonii
test sample size is: (66, 9)
*****
[24] [30]
real sample size is: (73, 9)
save_data/RealData_Seg_byName/Xu
test sample size is: (66, 9)
*****
[24] [31]
real sample size is: (73, 9)
save_data/RealData_Seg_byName/Yamamoto
test sample size is: (66, 9)
*****


In [16]:
print(rf_predicted)
print(svm_predicted)
print(decisiontree_predicted)
print(sum(rf_predicted) / len(rf_predicted))
print(sum(svm_predicted) / len(svm_predicted))
print(sum(decisiontree_predicted) / len(decisiontree_predicted))

[0.9090909090909091, 0.4090909090909091, 0.2878787878787879, 0.5, 0.5151515151515151, 0.3484848484848485, 0.5]
[0.9393939393939394, 0.6666666666666666, 0.07575757575757576, 0.3787878787878788, 0.19696969696969696, 0.48484848484848486, 0.5]
[0.5, 0.5454545454545454, 0.2727272727272727, 0.5454545454545454, 0.5757575757575758, 0.4090909090909091, 0.6212121212121212]
0.49567099567099565
0.4632034632034632
0.49567099567099565



ori
0.632034632034632
0.6147186147186147
0.5909090909090909

size3
0.70995670995671
0.6515151515151515
0.7056277056277055

size5
0.6731601731601732
0.6818181818181818
0.5995670995670996

size7
0.6731601731601732
0.6212121212121212
0.683982683982684

size5+7
0.6904761904761905
0.7337662337662338
0.6450216450216449

size3+5+7
0.7229437229437229
0.7402597402597404
0.6363636363636364

size3+7
0.7012987012987012
0.7034632034632036
0.7142857142857143

concave
0.6883116883116882
0.6688311688311688
0.670995670995671

convex
0.6450216450216449
0.6385281385281385
0.5584415584415584

inc
0.6991341991341992
0.6688311688311689
0.6623376623376622

dec
0.6580086580086579
0.6774891774891775
0.6450216450216449

all
